In [1]:
from sqlalchemy import create_engine, inspect
import pandas as pd

In [2]:
engine = create_engine("sqlite:///../db/mlsseasons.sqlite", echo=False)

In [3]:
inspect(engine)

In [4]:
dataset=pd.DataFrame(engine.execute("""
select s."First Name", s."Last Name", s.Position,
c.Club, c.pts, c."#" as standing, c.season
from seasons c
join club_map m on m.long_name = c.Club
join salaries s on s."Club (grouped)" = m.short_name
where c.season = s.Season
"""),columns=['First Name','Last Name','Position','Club','PTS','Standing','Season'])
dataset

,First Name,Last Name,Position,Club,PTS,Standing,Season
0,None,Carreiro da Silva Fred,M-F,DCD.C. United,55,1,2007
1,Ben,Olsen,M,DCD.C. United,55,1,2007
2,Brad,North,F,DCD.C. United,55,1,2007
3,Brian,Carroll,M,DCD.C. United,55,1,2007
4,Bryan,Arguez,M-D,DCD.C. United,55,1,2007
...,...,...,...,...,...,...,...
6206,Quincy,Amarikwa,F,SJSan Jose Earthquakes,21,23,2018
6207,Shea,Salinas,M,SJSan Jose Earthquakes,21,23,2018
6208,Tommy,Thompson,M,SJSan Jose Earthquakes,21,23,2018
6209,"Valeri ""Vako""",Qazaishvili,M,SJSan Jose Earthquakes,21,23,2018


In [18]:
engine.execute("select sqlite_version()").first()

('3.22.0',)

In [20]:
dataset["First Name"] = dataset["First Name"].fillna("")

In [21]:
dataset['Player'] = dataset['First Name'] + dataset['Last Name'] # + dataset['Position']
dataset['Team'] = dataset['Club'] + dataset['Season'].apply(str)
dataset

,First Name,Last Name,Position,Club,PTS,Standing,Season,Player,Team
0,,Carreiro da Silva Fred,M-F,DCD.C. United,55,1,2007,Carreiro da Silva Fred,DCD.C. United2007
1,Ben,Olsen,M,DCD.C. United,55,1,2007,BenOlsen,DCD.C. United2007
2,Brad,North,F,DCD.C. United,55,1,2007,BradNorth,DCD.C. United2007
3,Brian,Carroll,M,DCD.C. United,55,1,2007,BrianCarroll,DCD.C. United2007
4,Bryan,Arguez,M-D,DCD.C. United,55,1,2007,BryanArguez,DCD.C. United2007
...,...,...,...,...,...,...,...,...,...
6206,Quincy,Amarikwa,F,SJSan Jose Earthquakes,21,23,2018,QuincyAmarikwa,SJSan Jose Earthquakes2018
6207,Shea,Salinas,M,SJSan Jose Earthquakes,21,23,2018,SheaSalinas,SJSan Jose Earthquakes2018
6208,Tommy,Thompson,M,SJSan Jose Earthquakes,21,23,2018,TommyThompson,SJSan Jose Earthquakes2018
6209,"Valeri ""Vako""",Qazaishvili,M,SJSan Jose Earthquakes,21,23,2018,"Valeri ""Vako""Qazaishvili",SJSan Jose Earthquakes2018


In [43]:
# Example on one player
subset = dataset.loc[dataset["Player"] == "BenOlsen"].sort_values("Season")
subset["Experience"] = range(subset.count()["Season"])
subset
#dataset.loc[dataset["Player"] == "BenOlsen"]

,First Name,Last Name,Position,Club,PTS,Standing,Season,Player,Team,Experience
1,Ben,Olsen,M,DCD.C. United,55,1,2007,BenOlsen,DCD.C. United2007,0
627,Ben,Olsen,M,DCD.C. United,38,10,2008,BenOlsen,DCD.C. United2008,1
1005,Ben,Olsen,M,DCD.C. United,40,10,2009,BenOlsen,DCD.C. United2009,2


In [77]:
# Add a column to the dataset "Experience" - how many years the player was in MLS prior to the season
for player in pd.unique(dataset["Player"]):
    subset = dataset.loc[dataset["Player"] == player].sort_values("Season")
    subset["Experience"] = range(subset.count()["Season"])
    for season in subset["Season"]:
        exp = subset.loc[subset["Season"] == season]["Experience"].array[0]
        dataset.loc[((dataset["Player"] == player) & (dataset["Season"] == season)), ["Experience"]] = exp + 1
dataset


,First Name,Last Name,Position,Club,PTS,Standing,Season,Player,Team,Experience,encoded_player,present
0,,Carreiro da Silva Fred,M-F,DCD.C. United,55,1,2007,Carreiro da Silva Fred,DCD.C. United2007,1.0,384,1.0
1,Ben,Olsen,M,DCD.C. United,55,1,2007,BenOlsen,DCD.C. United2007,1.0,231,1.0
2,Brad,North,F,DCD.C. United,55,1,2007,BradNorth,DCD.C. United2007,1.0,279,1.0
3,Brian,Carroll,M,DCD.C. United,55,1,2007,BrianCarroll,DCD.C. United2007,1.0,311,1.0
4,Bryan,Arguez,M-D,DCD.C. United,55,1,2007,BryanArguez,DCD.C. United2007,1.0,331,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
6206,Quincy,Amarikwa,F,SJSan Jose Earthquakes,21,23,2018,QuincyAmarikwa,SJSan Jose Earthquakes2018,10.0,1812,1.0
6207,Shea,Salinas,M,SJSan Jose Earthquakes,21,23,2018,SheaSalinas,SJSan Jose Earthquakes2018,11.0,2014,1.0
6208,Tommy,Thompson,M,SJSan Jose Earthquakes,21,23,2018,TommyThompson,SJSan Jose Earthquakes2018,5.0,2122,1.0
6209,"Valeri ""Vako""",Qazaishvili,M,SJSan Jose Earthquakes,21,23,2018,"Valeri ""Vako""Qazaishvili",SJSan Jose Earthquakes2018,2.0,2165,1.0


In [59]:
dataset.groupby("Player").count()["Season"].describe()

count    2250.000000
mean        2.758667
std         2.360747
min         1.000000
25%         1.000000
50%         2.000000
75%         4.000000
max        12.000000
Name: Season, dtype: float64

In [60]:
y = dataset["Player"].tolist()

In [61]:
 from sklearn.preprocessing import LabelEncoder

# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y)
encoded_y = label_encoder.transform(y)
encoded_y

array([ 384,  231,  279, ..., 2122, 2165, 2222])

In [62]:
dataset['encoded_player'] = encoded_y
dataset['present'] = 1.0
dataset.loc[dataset['Player'] == 'BenOlsen']

,First Name,Last Name,Position,Club,PTS,Standing,Season,Player,Team,Experience,encoded_player,present
1,Ben,Olsen,M,DCD.C. United,55,1,2007,BenOlsen,DCD.C. United2007,0.0,231,1.0
627,Ben,Olsen,M,DCD.C. United,38,10,2008,BenOlsen,DCD.C. United2008,1.0,231,1.0
1005,Ben,Olsen,M,DCD.C. United,40,10,2009,BenOlsen,DCD.C. United2009,2.0,231,1.0


In [63]:
samples = dataset.pivot(index="Team", columns="encoded_player", values='present')
samples

encoded_player,0,1,2,3,4,5,6,7,8,9,...,2241,2242,2243,2244,2245,2246,2247,2248,2249,2250
Team,,,,,,,,,,,,,,,,,,,,,
ATLAtlanta United FC2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ATLAtlanta United FC2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CHIChicago Fire FC2007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CHIChicago Fire FC2008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CHIChicago Fire FC2009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VANVancouver Whitecaps FC2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
VANVancouver Whitecaps FC2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
VANVancouver Whitecaps FC2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
samples.describe()

encoded_player,0,1,2,3,4,5,6,7,8,9,...,2241,2242,2243,2244,2245,2246,2247,2248,2249,2250
count,2.0,10.0,4.0,1.0,2.0,1.0,4.0,3.0,1.0,5.0,...,2.0,1.0,2.0,6.0,1.0,1.0,1.0,2.0,2.0,4.0
mean,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
std,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0,...,0.0,NaN,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0
min,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
25%,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
50%,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
75%,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
max,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [78]:
experiences = dataset.pivot(index="Team", columns="encoded_player", values='Experience')
experiences

encoded_player,0,1,2,3,4,5,6,7,8,9,...,2241,2242,2243,2244,2245,2246,2247,2248,2249,2250
Team,,,,,,,,,,,,,,,,,,,,,
ATLAtlanta United FC2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ATLAtlanta United FC2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CHIChicago Fire FC2007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CHIChicago Fire FC2008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CHIChicago Fire FC2009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VANVancouver Whitecaps FC2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
VANVancouver Whitecaps FC2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
VANVancouver Whitecaps FC2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:
experiences.describe()

encoded_player,0,1,2,3,4,5,6,7,8,9,...,2241,2242,2243,2244,2245,2246,2247,2248,2249,2250
count,2.000000,10.00000,4.000000,1.0,2.000000,1.0,4.000000,3.0,1.0,5.000000,...,2.000000,1.0,2.000000,6.000000,1.0,1.0,1.0,2.000000,2.000000,0.0
mean,1.500000,5.50000,2.500000,1.0,1.500000,1.0,2.500000,2.0,1.0,3.000000,...,1.500000,1.0,1.500000,3.500000,1.0,1.0,1.0,1.500000,1.500000,NaN
std,0.707107,3.02765,1.290994,NaN,0.707107,NaN,1.290994,1.0,NaN,1.581139,...,0.707107,NaN,0.707107,1.870829,NaN,NaN,NaN,0.707107,0.707107,NaN
min,1.000000,1.00000,1.000000,1.0,1.000000,1.0,1.000000,1.0,1.0,1.000000,...,1.000000,1.0,1.000000,1.000000,1.0,1.0,1.0,1.000000,1.000000,NaN
25%,1.250000,3.25000,1.750000,1.0,1.250000,1.0,1.750000,1.5,1.0,2.000000,...,1.250000,1.0,1.250000,2.250000,1.0,1.0,1.0,1.250000,1.250000,NaN
50%,1.500000,5.50000,2.500000,1.0,1.500000,1.0,2.500000,2.0,1.0,3.000000,...,1.500000,1.0,1.500000,3.500000,1.0,1.0,1.0,1.500000,1.500000,NaN
75%,1.750000,7.75000,3.250000,1.0,1.750000,1.0,3.250000,2.5,1.0,4.000000,...,1.750000,1.0,1.750000,4.750000,1.0,1.0,1.0,1.750000,1.750000,NaN
max,2.000000,10.00000,4.000000,1.0,2.000000,1.0,4.000000,3.0,1.0,5.000000,...,2.000000,1.0,2.000000,6.000000,1.0,1.0,1.0,2.000000,2.000000,NaN


In [67]:
dataset['Standing'] = dataset['Standing'].astype(int)
dataset['PTS'] = dataset['PTS'].astype(int)
y = dataset.groupby('Team').mean()['Standing'].sort_index()
y

Team
ATLAtlanta United FC2017          4.0
ATLAtlanta United FC2018          2.0
CHIChicago Fire FC2007            8.0
CHIChicago Fire FC2008            3.0
CHIChicago Fire FC2009            6.0
                                 ... 
VANVancouver Whitecaps FC2014     9.0
VANVancouver Whitecaps FC2015     3.0
VANVancouver Whitecaps FC2016    16.0
VANVancouver Whitecaps FC2017     8.0
VANVancouver Whitecaps FC2018    14.0
Name: Standing, Length: 218, dtype: float64

In [68]:
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(1)
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import sklearn.datasets

In [80]:
X = samples.join(experiences, rsuffix="e").fillna(0).sort_index()
X

encoded_player,0,1,2,3,4,5,6,7,8,9,...,2241e,2242e,2243e,2244e,2245e,2246e,2247e,2248e,2249e,2250e
Team,,,,,,,,,,,,,,,,,,,,,
ATLAtlanta United FC2017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ATLAtlanta United FC2018,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CHIChicago Fire FC2007,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CHIChicago Fire FC2008,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CHIChicago Fire FC2009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VANVancouver Whitecaps FC2014,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
VANVancouver Whitecaps FC2015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
VANVancouver Whitecaps FC2016,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [141]:
# Use train_test_split to create training and testing data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=3)

In [82]:
X_test.shape

(55, 4502)

In [127]:
# first, create a normal neural network with 2 inputs, 6 hidden nodes, and 2 outputs
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(units=32, activation='relu', input_dim=X_train.shape[-1]))
model.add(Dense(units=16, activation='relu'))
model.add(Dense(units=8, activation='relu'))
model.add(Dense(units=1))
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_40 (Dense)             (None, 32)                144096    
_________________________________________________________________
dense_41 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_42 (Dense)             (None, 8)                 136       
_________________________________________________________________
dense_43 (Dense)             (None, 1)                 9         
Total params: 144,769
Trainable params: 144,769
Non-trainable params: 0
_________________________________________________________________


In [128]:
import tensorflow as tf
# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['accuracy'])

In [129]:
# Fit the model to the training data
model.fit(
    X_train,
    y_train,
    epochs=20,
    shuffle=True,
    verbose=1
)

Epoch 1/20
6/6 [==============================] - 0s 5ms/step - loss: 123.7185 - accuracy: 0.0000e+00
Epoch 2/20
6/6 [==============================] - 0s 4ms/step - loss: 120.7649 - accuracy: 0.0000e+00
Epoch 3/20
6/6 [==============================] - 0s 4ms/step - loss: 116.8655 - accuracy: 0.0184
Epoch 4/20
6/6 [==============================] - 0s 3ms/step - loss: 109.8531 - accuracy: 0.0307
Epoch 5/20
6/6 [==============================] - 0s 4ms/step - loss: 99.8947 - accuracy: 0.0613
Epoch 6/20
6/6 [==============================] - 0s 5ms/step - loss: 86.4110 - accuracy: 0.0613
Epoch 7/20
6/6 [==============================] - 0s 4ms/step - loss: 72.4305 - accuracy: 0.0613
Epoch 8/20
6/6 [==============================] - 0s 3ms/step - loss: 58.2571 - accuracy: 0.0613
Epoch 9/20
6/6 [==============================] - 0s 4ms/step - loss: 45.0211 - accuracy: 0.0613
Epoch 10/20
6/6 [==============================] - 0s 4ms/step - loss: 33.9585 - accuracy: 0.0613
Epoch 11/20
6/6 [

In [131]:
model_loss, model_accuracy = model.evaluate(
    X_test, y_test, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

2/2 - 0s - loss: 28.3667 - accuracy: 0.0364
Normal Neural Network - Loss: 28.366748809814453, Accuracy: 0.036363635212183


In [145]:
model.predict(X_test.head(1))

array([[13.417073]], dtype=float32)

In [142]:
y_test.head(1)

Team
DCD.C. United2016    10.0
Name: Standing, dtype: float64